## <p style="color:red;text-align:center">**News Classifier**</p>

### <p style="text-align:center">**ML Pipeline with Sentence Transformers, ChromaDB & Airflow**</p>

Ce projet vise à construire un pipeline complet de classification de textes permettant de catégoriser automatiquement des articles d’actualité dans 4 classes :

- World
- Sports
- Business
- Sci/Tech

Le pipeline utilise des techniques NLP avancées, des embeddings vectoriels, une base de données vectorielle et une orchestration via Apache Airflow.

<br>

### <span style="color:green">**Stockage dans ChromaDB :**</span>

#### <span style="color:orange">**1. Charger les Données :**</span>

In [2]:
import pandas as pd

df_train = pd.read_pickle("../metadatas/train.pkl")

print("Train Dataset Chargée avec Succès !")

df_train.head()

Train Dataset Chargée avec Succès !


,text,clean_text,label,text_embedding,id
0,wall st. bears claw back into the black (reute...,wall bears claw back black reuters reuters wal...,2,"[0.08452066034078598, -0.010081029497087002, 0...",train_art_0
1,carlyle looks toward commercial aerospace (reu...,carlyle looks toward commercial aerospace reut...,2,"[0.05757696554064751, -0.04762953147292137, -0...",train_art_1
2,oil and economy cloud stocks' outlook (reuters...,oil economy cloud stocks outlook reuters reute...,2,"[0.007693152409046888, -0.05420009419322014, 0...",train_art_2
3,iraq halts oil exports from main southern pipe...,iraq halts oil exports main southern pipeline ...,2,"[0.0023386478424072266, -0.016605667769908905,...",train_art_3
4,"oil prices soar to all-time record, posing new...",oil prices soar record posing new menace us ec...,2,"[-0.0641697347164154, -0.018890680745244026, 0...",train_art_4


In [3]:
import pandas as pd

df_test = pd.read_pickle("../metadatas/test.pkl")

print("Test Dataset Chargée avec Succès !")

df_test.head()

Test Dataset Chargée avec Succès !


,text,clean_text,label,text_embedding,id
0,fears for t n pension after talks unions repre...,fears n pension talks unions representing work...,2,"[-0.014112311415374279, 0.150861456990242, -0....",test_art_0
1,the race is on: second private team sets launc...,race second private team sets launch date huma...,3,"[-0.010881485417485237, 0.06553682684898376, -...",test_art_1
2,ky. company wins grant to study peptides (ap) ...,company wins grant study peptides ap ap compan...,3,"[-0.07935794442892075, -0.0015284559922292829,...",test_art_2
3,prediction unit helps forecast wildfires (ap) ...,prediction unit helps forecast wildfires ap ap...,3,"[-0.07381721585988998, 0.0071342745795845985, ...",test_art_3
4,calif. aims to limit farm-related smog (ap) ap...,aims limit smog ap ap southern california agen...,3,"[0.010134254582226276, 0.014008638449013233, 0...",test_art_4


#### <span style="color:orange">**2. Installer & configurer `ChromaDB` :**</span>

`ChromaDB` est une **base de données vectorielle** open-source et performante, conçue spécifiquement pour servir de "mémoire à long terme" aux applications d'intelligence artificielle modernes. 

Contrairement aux bases de données classiques qui stockent du texte brut et cherchent des mots-clés exacts, `ChromaDB` stocke des **embeddings** (des vecteurs numériques représentant le sens sémantique d'un texte, d'une image ou d'un son) ainsi que leurs métadonnées associées. 

Son utilité principale réside dans sa capacité à effectuer des **recherches de similarité** ultra-rapides : elle permet à un modèle d'IA de retrouver instantanément les informations les plus pertinentes contextuellement par rapport à une requête donnée, ce qui la rend indispensable pour alimenter des Chatbots (RAG), des systèmes de recommandation ou des moteurs de recherche sémantique.

Pour installer ChromaDB, on utilise simplement :

```bash
pip install chromadb
```

In [4]:
import chromadb

print("Version :", chromadb.__version__)

Version : 1.3.6


#### <span style="color:orange">**3. Créer les Collections :**</span>

In [5]:
client = chromadb.PersistentClient(path="../chroma_db")

collection_train = client.create_collection(name="train_news", metadata={"hnsw:space" : "cosine"})
collection_test = client.create_collection(name="test_news", metadata={"hnsw:space" : "cosine"})

#### <span style="color:orange">**4. Insérer Embeddings, Identifiants et Labels :**</span>

In [17]:
batch_size = 5000

for i in range(0, len(df_train), batch_size):
    batch = df_train.iloc[i:i + batch_size]

    batch_ids = batch["id"].astype(str).tolist()
    batch_embeddings = batch["text_embedding"].tolist()
    batch_metadata = batch[["label", "text"]].to_dict("records")

    collection_train.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        metadatas=batch_metadata
    )

print(f"Nombre des Items dans la Collection d'Entraînement : {collection_train.count()}")

Nombre des Items dans la Collection d'Entraînement : 120000


In [18]:
batch_size = 5000

for i in range(0, len(df_test), batch_size):
    batch = df_test.iloc[i:i + batch_size]

    batch_ids = batch["id"].astype(str).tolist()
    batch_embeddings = batch["text_embedding"].tolist()
    batch_metadata = batch[["label", "text"]].to_dict("records")

    collection_test.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        metadatas=batch_metadata
    )

print(f"Nombre des Items dans la Collection de Test : {collection_test.count()}")

Nombre des Items dans la Collection de Test : 7600


In [20]:
train_item = collection_train.peek(limit=1)

print("\nTrain Item : ")
print(f"- ID: {train_item['ids'][0]}")
print(f"- Label: {train_item['metadatas'][0]['label']}")
print(f"- Text: {train_item['metadatas'][0]['text'][:50]}...")
print(f"- Embedding : {train_item['embeddings'][0][:10]}...")
print(f"- Embedding length: {len(train_item['embeddings'][0])}")


Train Item : 
- ID: train_art_0
- Label: 2
- Text: wall st. bears claw back into the black (reuters) ...
- Embedding : [ 0.08452066 -0.01008103  0.06637681  0.04122576  0.02236528 -0.03327266
 -0.05002487 -0.11134939 -0.01814673 -0.00200442]...
- Embedding length: 384


In [21]:
test_item = collection_test.peek(limit=1)

print("\Test Item : ")
print(f"- ID: {test_item['ids'][0]}")
print(f"- Label: {test_item['metadatas'][0]['label']}")
print(f"- Text: {test_item['metadatas'][0]['text'][:50]}...")
print(f"- Embedding : {test_item['embeddings'][0][:10]}...")
print(f"- Embedding length: {len(test_item['embeddings'][0])}")

\Test Item : 
- ID: test_art_0
- Label: 2
- Text: fears for t n pension after talks unions represent...
- Embedding : [-0.01411231  0.15086146 -0.02255305  0.02576183  0.04102316  0.00325697
 -0.04564526  0.04752721  0.04816809 -0.08691055]...
- Embedding length: 384


<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
C:\Users\abdel\AppData\Local\Temp\ipykernel_14444\3887453073.py:3: SyntaxWarning: invalid escape sequence '\T'
  print("\Test Item : ")
